# <strong><font color=#A52A2A></span> NLP_HW4 </strong>
Dr Asgari - Spring 2023
<br>
98101566 -> Mohammadreza Daviran
<br>
98170668 -> Sara Azarnoush
<br>
98171007 -> Nona Ghazizadeh
<br>
<strong><font color=#A52A2A>Token classification

# <font color='red'> Table of content
-------

 - Intoduction
 - Requirements
    - Download
    - Import
 - Download and load
   - Load dataset
   - Example
 - Create dataset
 - Preprocess
  - Preprocess
  - Save
    - Train
    - Validation
    - Test
 - Read data
 - Labels
 - TF_IDF  
    - QADataset
    - TF_IDF
 - HMM
    - Vectorization
    - Model
    - Evaluation
        - F1 score
        - EM score
 - LSTM/CRF model
    - Requirements
    - Model
      - LSTM/CRF
      - Simple LSTM
      - Padding labels
    - Training
      - Training with training data
      - Training with training and validation data
      - Evaluation

 - Transformer

# <font color='red'> Introduction </font>
-------

In this project, we Implement a model for extractive question and answer.

It inputs a text and question and returns an answer from the text.

We import project requirements and download the SajjadAyoubi/persian_qa dataset and preprocess it and save clean data in JSON and read it.

We make labels from the start index and end index of the answer in text and vectorized them with tf-idf and make a  new dataset.

More information is available in each section.

# <font color='red'> Requirements </font>
-------

Download and import requirements for this project.

## <font color='green'> Download </font>

Download project requirements

*note: If you get an error uninstall and install numpy then restart the execution and run again.

In [ ]:
# !pip uninstall numpy

In [ ]:
!pip install numpy

In [ ]:
! pip install tqdm
! pip install gdown
! pip install pandas
! pip install transformers
! pip install torch
! pip install datasets
! pip install scikit-learn
! pip install transformers
! pip install hazm

## <font color='green'> Import requirements </font>

Import project requirements

In [ ]:
import re
import hazm
import json
import gdown
import torch
import pickle
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from collections import OrderedDict
from pathlib import Path

import pandas as pd
from tqdm import tqdm
from tabulate import tabulate
from string import punctuation
from transformers import AutoTokenizer

from sklearn.naive_bayes import MultinomialNB
from transformers import DataCollatorWithPadding


from sklearn.linear_model import LogisticRegression
from transformers import BigBirdModel, AutoTokenizer
from sklearn.feature_extraction.text import TfidfVectorizer
from transformers import Trainer, TrainingArguments, AutoModelForSequenceClassification

from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix

In [ ]:

from datasets import DatasetDict, Dataset
from datasets import load_dataset

# <font color='red'> Download and load </font>
------

Downlad "SajjadAyoubi/persian_qa"  dataset.


## <font color='green'> Load dataset </font>

For this task we use "SajjadAyoubi/persian_qa" dataset.

In [ ]:
dataset = load_dataset("SajjadAyoubi/persian_qa")

  0%|          | 0/2 [00:00<?, ?it/s]

## <font color='green'> Example </font>

In [ ]:
dataset['train'][1]

{'id': 2,
 'title': 'شرکت فولاد مبارکه اصفهان',
 'context': 'شرکت فولاد مبارکۀ اصفهان، بزرگ\u200cترین واحد صنعتی خصوصی در ایران و بزرگ\u200cترین مجتمع تولید فولاد در کشور ایران است، که در شرق شهر مبارکه قرار دارد. فولاد مبارکه هم\u200cاکنون محرک بسیاری از صنایع بالادستی و پایین\u200cدستی است. فولاد مبارکه در ۱۱ دوره جایزۀ ملی تعالی سازمانی و ۶ دوره جایزۀ شرکت دانشی در کشور رتبۀ نخست را بدست آورده\u200cاست و همچنین این شرکت در سال ۱۳۹۱ برای نخستین\u200cبار به عنوان تنها شرکت ایرانی با کسب امتیاز ۶۵۴ تندیس زرین جایزۀ ملی تعالی سازمانی را از آن خود کند. شرکت فولاد مبارکۀ اصفهان در ۲۳ دی ماه ۱۳۷۱ احداث شد و اکنون بزرگ\u200cترین واحدهای صنعتی و بزرگترین مجتمع تولید فولاد در ایران است. این شرکت در زمینی به مساحت ۳۵ کیلومتر مربع در نزدیکی شهر مبارکه و در ۷۵ کیلومتری جنوب غربی شهر اصفهان واقع شده\u200cاست. مصرف آب این کارخانه در کمترین میزان خود، ۱٫۵٪ از دبی زاینده\u200cرود برابر سالانه ۲۳ میلیون متر مکعب در سال است و خود یکی از عوامل کم\u200cآبی زاینده\u200cرود شناخته می\u200cشود.',
 'questio

In [ ]:
dataset['train'][1]['context'].find("۶")

263

In [ ]:
for i, x in enumerate(dataset['train']):
  if i == 1:
    break
  print(x['answers'])

{'text': ['در شرق شهر مبارکه'], 'answer_start': [114]}


# <font color='red'> Creating Dataset </font>
-----

Create new proper train, test and validation dataset.

In [ ]:
splitter = int(0.85 * len(dataset['train']))
splitter

5360

We set text, question, answer, the first index of the answer, and dic with x and y for train, test, and validation datasets. First, we set text, question, answer, and start, then concat text and answer for x value and first index and last index of answer for y value. \\
At first, we calculate the splitter and then create data if I < splitter set them as our train value, else we set them as validation data. Set our test dataset from the validation dataset.

In [ ]:
train_val_contexts = []
train_val_questions = []
train_val_answers = []
train_val_start = []
train_val_data = {"x": [], "y": []}

train_contexts = []
train_questions = []
train_answers = []
train_start = []
train_data = {"x": [], "y": []}

valid_contexts = []
valid_questions = []
valid_answers = []
valid_start = []
valid_data = {"x": [], "y": []}

test_contexts = []
test_questions = []
test_answers = []
test_start = []
test_data = {"x": [], "y": []}


In [ ]:
for i, x in enumerate(dataset['train']):
  try:
      context = x['context']
      question = x['question']
      answer = x['answers']['text'][0]
      start = x['answers']['answer_start'][0]

      train_val_data['x'].append(context + question)
      train_val_data['y'].append(answer)
  except:
      continue

  if i < splitter:
    context = x['context']
    question = x['question']
    answer = x['answers']['text'][0]
    start = x['answers']['answer_start'][0]

    train_contexts.append(context)
    train_questions.append(question)
    train_answers.append(answer)
    train_start.append(start)

    train_data['x'].append(context + question)
    train_data['y'].append((start, start + len(answer)))

  else:
    context = x['context']
    question = x['question']
    answer = x['answers']['text'][0]
    start = x['answers']['answer_start'][0]

    valid_answers.append(x['answers']['text'][0])
    valid_contexts.append(x['context'])
    valid_questions.append(x['question'])
    valid_start.append(x['answers']['answer_start'][0])

    valid_data['x'].append(context + question)
    valid_data['y'].append((start, start + len(answer)))


for i, x in enumerate(dataset['validation']):
  try:
    context = x['context']
    question = x['question']
    answer = x['answers']['text'][0]
    start = x['answers']['answer_start'][0]

    test_answers.append(x['answers']['text'][0])
    test_contexts.append(x['context'])
    test_questions.append(x['question'])
    test_start.append(x['answers']['answer_start'][0])

    test_data['x'].append(context + question)
    test_data['y'].append((start, start + len(answer)))
  except:
      continue

# <font color='red'> Preprocess </font>
----

Preprocess data to get clean data.

## <font color='green'> Preprocess </font>

Get data to normalize it, remove links and punctuations, and tokenize and remove chosen stop words from it.

In [ ]:
class Preprocessor:

    def __init__(self, stopwords_path):
        self.stopwords = []
        with open(stopwords_path, encoding='utf-8') as file:
            self.stopwords = file.read().split()

    def preprocess(self, text):
        text = self.normalize(text)
        text = self.remove_links(text)
        text = self.remove_punctuations(text)
        words = self.word_tokenize(text)
        words = self.remove_stopwords(words)
        return words

    def normalize(self, text):
        return hazm.Normalizer().normalize(text)

    def remove_links(self, text):
        patterns = ['\S*http\S*', '\S*www\S*', '\S+\.ir\S*', '\S+\.com\S*', '\S+\.org\S*', '\S*@\S*']
        for pattern in patterns:
            text = re.sub(pattern, ' ', text)
        return text

    def remove_punctuations(self, text):
        return re.sub(f'[{punctuation}؟،٪×÷»«]+', '', text)

    def word_tokenize(self, text):
        return hazm.word_tokenize(text)

    def remove_invalid_words(self, words):
        return [word for word in words if len(word) > 3 or re.match('^[\u0600-\u06FF]{2,3}$', word)]

    def remove_stopwords(self, words):
        return [word for word in words if word not in self.stopwords]

In [ ]:
def get_preprocessed_text(data, preprocessor):
    preprocessed_contents = []
    contents = data
    preprocessed_contents = [preprocessor.preprocess(content) for content in tqdm(contents)]
    return preprocessed_contents

Downlaod stopwords from google drive

In [ ]:
url = "https://drive.google.com/file/d/1OVWfL-UubaJn-bBKW3s4CD3JrM_84mYj"
output = "stopwords.txt"
gdown.download(url, output, quiet=False)

/usr/local/lib/python3.10/dist-packages/gdown/parse_url.py:35: UserWarning: You specified a Google Drive link that is not the correct link to download a file. You might want to try `--fuzzy` option or the following url: https://drive.google.com/uc?id=None
  warnings.warn(
Downloading...
From: https://drive.google.com/file/d/1OVWfL-UubaJn-bBKW3s4CD3JrM_84mYj
To: /content/stopwords.txt
80.2kB [00:00, 8.77MB/s]


'stopwords.txt'

In [ ]:
preprocessor = Preprocessor(stopwords_path='stopwords.txt')

## <font color='green'> Save </font>

Save text and answer for all three trains, validation, and test datasets after preprocessing them in JSON file.

### <font color='blue'> Train

Save text and answer for trains datasets after preprocessing them in JSON file.

In [ ]:
train_preprocessed_content = get_preprocessed_text(train_data['x'], preprocessor)

In [ ]:
train_preprocessed_answers = get_preprocessed_text(train_answers, preprocessor)

In [ ]:
with open('token_preprocessed_train_dataset.json', 'w') as f:
    json.dump(train_preprocessed_content, f)

with open('token_preprocessed_train_answers_dataset.json', 'w') as f:
    json.dump(train_preprocessed_answers, f)

### <font color='blue'> Validation

Save text and answer for validation datasets after preprocessing them in JSON file.

In [ ]:
val_preprocessed_content = get_preprocessed_text(valid_data['x'], preprocessor)

In [ ]:
val_preprocessed_answers = get_preprocessed_text(valid_answers, preprocessor)

In [ ]:
with open('token_preprocessed_val_dataset.json', 'w') as f:
    json.dump(val_preprocessed_content, f)

with open('token_preprocessed_val_answers_dataset.json', 'w') as f:
    json.dump(val_preprocessed_answers, f)

### <font color='blue'> Test

Save text and answer for test datasets after preprocessing them in JSON file.

In [ ]:
test_preprocessed_content = get_preprocessed_text(test_data['x'], preprocessor)

In [ ]:
test_preprocessed_answers = get_preprocessed_text(test_answers, preprocessor)

In [ ]:
with open('token_preprocessed_test_dataset.json', 'w') as f:
    json.dump(test_preprocessed_content, f)

with open('token_preprocessed_test_answers_dataset.json', 'w') as f:
    json.dump(test_preprocessed_answers, f)

# <font color='red'> Read data </font>
------

Download and read processed data from google drive

In [ ]:
import gdown
import json
import numpy as np

In [ ]:
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

Download text and answer dataset for train, test and validation from google drive.

In [ ]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)
# train
download = drive.CreateFile({'id': '1BYQTIAvaCbxUlHhYYDTaVvJmPyf9hX6T'})
download.GetContentFile('token_preprocessed_train_dataset.json')
download = drive.CreateFile({'id': '1D--9Mv22LzjXC8T59ZOEXTSzI6AqmC2Y'})
download.GetContentFile('token_preprocessed_train_answers_dataset.json')
# validation
download = drive.CreateFile({'id': '1QIKf2etnVYIywhoWSRAmnj6SJAJ_fuNq'})
download.GetContentFile('token_preprocessed_val_dataset.json')
download = drive.CreateFile({'id': '19ft10_swfFs9TI1qH9ZTPDbFm90Xp76Y'})
download.GetContentFile('token_preprocessed_val_answers_dataset.json')
# test
download = drive.CreateFile({'id': '1_PxwHovDumnssLJZAWqa6JcmOoNKRbLC'})
download.GetContentFile('token_preprocessed_test_dataset.json')
download = drive.CreateFile({'id': '1Ssd90fwrklFD0wiOnj6HpnYtxwJYmiBP'})
download.GetContentFile('token_preprocessed_test_answers_dataset.json')

Read and load json file

In [ ]:
# train
with open('token_preprocessed_train_dataset.json', 'r') as f:
    train_preprocessed_content = json.load(f)
with open('token_preprocessed_train_answers_dataset.json', 'r') as f:
    train_preprocessed_answers = json.load(f)
# validation
with open('token_preprocessed_val_dataset.json', 'r') as f:
    val_preprocessed_content = json.load(f)
with open('token_preprocessed_val_answers_dataset.json', 'r') as f:
    val_preprocessed_answers = json.load(f)
# test
with open('token_preprocessed_test_dataset.json', 'r') as f:
    test_preprocessed_content = json.load(f)
with open('token_preprocessed_test_answers_dataset.json', 'r') as f:
    test_preprocessed_answers = json.load(f)

# <font color='red'> Labels </font>
-----

Concat processed data and create lables from start index and end index of answer in text.


Search content train dataset, concat each of them answer dataset if the answer was in content save start and end index of answer in labels.

In [ ]:
train_labels = []
for i, d in enumerate(train_preprocessed_content):
    length = len(train_preprocessed_answers[i])
    answer = " ".join(train_preprocessed_answers[i])
    start_index = 0
    end_index = 0

    labels = np.zeros(len(d))
    for i in range(len(d)):
        substr = " ".join(d[i:i+length])
        if answer.strip() == substr.strip():
            start_index = i
            end_index = i + length
            break
    labels[start_index:end_index] = 1
    train_labels.append(labels)

print(train_preprocessed_content[0][start_index:end_index])
print(answer)
len(train_labels)

5361


Search content validation dataset, concat each of them answer dataset if the answer was in content save start and end index of answer in labels.

In [ ]:
val_labels = []
for i, d in enumerate(val_preprocessed_content):
    length = len(val_preprocessed_answers[i])
    answer = " ".join(val_preprocessed_answers[i])
    start_index = 0
    end_index = 0

    labels = np.zeros(len(d))
    for i in range(len(d)):
        substr = " ".join(d[i:i+length])
        if answer.strip() == substr.strip():
            start_index = i
            end_index = i + length
            break
    labels[start_index:end_index] = 1
    val_labels.append(labels)
len(val_labels)

945


Search content test dataset, concat each of them answer dataset if the answer was in content save start and end index of answer in labels.

In [ ]:
test_labels = []
for i, d in enumerate(test_preprocessed_content):
    length = len(test_preprocessed_answers[i])
    answer = " ".join(test_preprocessed_answers[i])
    start_index = 0
    end_index = 0

    labels = np.zeros(len(d))
    for i in range(len(d)):
        substr = " ".join(d[i:i+length])
        if answer.strip() == substr.strip():
            start_index = i
            end_index = i + length
            break
    labels[start_index:end_index] = 1
    test_labels.append(labels)
len(test_labels)

651

# <font color='red'> TF_IDF </font>
------

Vectorize train, validation, and test content with TF-IDF model.

## <font color='green'> QADataset

Create new QA dataset

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer

from datasets import DatasetDict, Dataset
from datasets import load_dataset

In [ ]:
class QADataset(Dataset):
    def __init__(self, docs, labels):
        self.documents = docs
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        document = self.documents[idx]
        label = self.labels[idx]
        return document, label

## <font color='green'> TF_IDF

Vectorize train-val, and test content with TF-IDF model.

In [ ]:
from torch.utils.data import DataLoader, Dataset

In [ ]:
class TF_IDF:
    def __init__(self):
        self.vectorizer = TfidfVectorizer()

    def fit_vectorizer(self, data):
        self.vectorizer.fit(list(map(lambda doc: ' '.join(doc), data)))

    def transform_vectorizer(self, preprocessed_content):
        embeddings = []
        for content in preprocessed_content:
            emb_content = self.vectorizer.transform([' '.join(content)]).toarray()[0]
            embeddings.append(emb_content)
        return embeddings

Concat preprocessed data and vectorized it with tf-idf model.

In [ ]:
all_preprocessed_content = train_preprocessed_content + val_preprocessed_content + test_preprocessed_content

In [ ]:
TF_IDF_model = TF_IDF()
TF_IDF_model.fit_vectorizer(all_preprocessed_content)

training_vectors = np.array(TF_IDF_model.transform_vectorizer(train_preprocessed_content))
validation_vectors = np.array(TF_IDF_model.transform_vectorizer(val_preprocessed_content))
test_vectors = np.array(TF_IDF_model.transform_vectorizer(test_preprocessed_content))

In [ ]:
print(training_vectors.shape[0])
print(len(train_labels))
print(validation_vectors.shape[1])
print(test_vectors.shape[1])

5361
5361
19151
19151


In [ ]:
# Assuming you have 'train_data' and 'valid_data' containing your training and validation data respectively
train_dataset = QADataset(training_vectors, train_labels)
valid_dataset = QADataset(validation_vectors, val_labels)
test_dataset = QADataset(test_vectors, test_labels)

batch_size = 16
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
valid_loader = DataLoader(valid_dataset, batch_size=batch_size)
test_loader = DataLoader(test_dataset, batch_size=batch_size)

# <font color='red'> HMM </font>
------
A simple model for Extractive Question Answering could be a HMM model that gets a input as a concatenation of the text and the question together. Also the observation for each word will be the parts of the document which are in the answer too.

For this matter first we install hmmlearn library.

In [ ]:
!pip install hmmlearn

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 160.4/160.4 kB 5.6 MB/s eta 0:00:00


Then we import hmm model from hmmlearn library:

In [ ]:
from hmmlearn import hmm

We create data from concatenation of train data and validation data. Then we use this data to fit the model on it.

In [ ]:
train_val_data = train_preprocessed_content + val_preprocessed_content
train_val_answers = train_preprocessed_answers + val_preprocessed_answers

## <font color='green'> TF_IDF </font>

Vectorize train-val, and test content with TF-IDF model.

In [ ]:
TF_IDF_model_hmm = TF_IDF()

TF_IDF_model_hmm = TF_IDF()
TF_IDF_model_hmm.fit_vectorizer(train_val_data[20:50])

training_vectors = np.array(TF_IDF_model_hmm.transform_vectorizer(train_val_data))

training_vectors_ans = np.array(TF_IDF_model_hmm.transform_vectorizer(train_val_answers))

X_train = np.concatenate((training_vectors, training_vectors_ans), axis=1)

In [ ]:
X_train.shape

(6306, 1032)

## <font color='green'> Model </font>
We fit our model on the X_train which is a concatenation of the training_vectors and training_vectors_ans.

In [ ]:
hmm2 = hmm.GaussianHMM(n_components=2)
hmm2.startprob_ = np.ones(2) / 2
hmm2.transmat_ = np.ones((2, 2)) / 2

In [ ]:
X_train = np.round(X_train).astype(int)
X_train = np.nan_to_num(X_train)
hmm2.fit(X_train)

GaussianHMM(n_components=2)

In [ ]:
test_vectors = np.array(TF_IDF_model_hmm.transform_vectorizer(test_preprocessed_content))
test_vectors_ans = np.array(TF_IDF_model_hmm.transform_vectorizer(test_preprocessed_answers))
X_test = np.concatenate((test_vectors, test_vectors_ans), axis=1)

In [ ]:
hmm2.transmat_ += np.eye(hmm2.n_components) * 1e-10
hmm2.transmat_ /= hmm2.transmat_.sum(axis=1, keepdims=True)

In [ ]:
hmm2.transmat_

array([[2.00000000e-10, 1.00000000e+00],
       [1.13891609e-02, 9.88610839e-01]])

In [ ]:
X_test[0]

array([0., 0., 0., ..., 0., 0., 0.])

## <font color='green'> Evaluation </font>

Compare true lables and predicted labels to find EM and f1.

In [ ]:
def evaluate_model(model, test_data, test_answers):
    predictions = []
    for i in range(len(test_data[:5])):

        test_vectors = np.array(TF_IDF_model_hmm.transform_vectorizer([' '.join(test_data[i])]))
        test_vectors_ans = np.array(TF_IDF_model_hmm.transform_vectorizer([' '.join(test_answers[i])]))
        X_test = np.concatenate((test_vectors, test_vectors_ans), axis=1)
        X_test = np.round(X_test).astype(int)
        X_test = np.nan_to_num(X_test)
        pred = model.predict(X_test)
        print(pred)
        predictions.append(pred)
    return predictions

predictions = evaluate_model(hmm2, test_preprocessed_content, test_preprocessed_answers)

In [ ]:
hmm_pred = []
for idx, i in enumerate(X_test):
    hmm_pred.append(hmm2.predict(X_test[idx].reshape(-1,1)))

In [ ]:
reshaped_test_labels = []
for idx, i in enumerate(test_labels):
    label_reshape = np.resize(test_labels[idx], (1032,))
    reshaped_test_labels.append(label_reshape)

from evaluate we import load which ables us to use squad_metric. This metric can calculate EM and f1.

### <font color='blue'> F1 score </font>

we define f1_score function to calculate f1_score according to the prediction and ground_truth.

In [ ]:
def f1_score(prediction, ground_truth):
    prediction = np.array(prediction)
    ground_truth = np.array(ground_truth)
    tp = np.sum(prediction * ground_truth)
    fp = np.sum(prediction * (1 - ground_truth))
    fn = np.sum((1 - prediction) * ground_truth)
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    f1 = 2 * precision * recall / (precision + recall)
    return f1

f1_score(hmm_pred, reshaped_test_labels)

0.06163341644009166

### <font color='blue'> EM score </font>

Also the exact_match_score has been defined to calculate the EM metric according to the prediction and ground_truth.

In [ ]:
def exact_match_score(prediction, ground_truth):
    prediction = np.array(prediction)
    ground_truth = np.array(ground_truth)
    return np.sum(np.all(prediction == ground_truth, axis=1)) / prediction.shape[0]
exact_match_score(hmm_pred, reshaped_test_labels)

0.0

# <font color='red'> LSTM/CRF model </font>
------
For this model we do the same thing as HMM model with a few differences.

## <font color='green'> Requirements </font>

First we install torchcrf library and import necessary libraries and functions.

In [ ]:
! pip install torchcrf

In [ ]:
! pip install --upgrade torchcrf

In [ ]:
! pip install pytorch-crf

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from TorchCRF import CRF

## <font color='green'> Model </font>

We use LSTM and CRF models to build a LSTM/CRF model.

### <font color='blue'> LSTM/CRF </font>

This class provides a convenient way to define and use an LSTM-CRF model for sequence labeling tasks, where the LSTM layer is used to capture the sequential information, and the CRF layer helps to model the dependencies among the labels in the sequence.

In [ ]:
class LSTMCRFModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_labels):
        super(LSTMCRFModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, batch_first=True)
        self.crf = CRF(num_labels)

    def forward(self, x, labels, mask):
        lstm_output, _ = self.lstm(x)
        crf_output = self.crf(lstm_output, labels, mask=mask)
        return crf_output

### <font color='blue'> Simple LSTM </font>

Another approach to solve Extractive Question Answering is to use LSTMModel with fully-connected layers. The LSTM layer allows the model to capture the contextual information and dependencies within the input sequence. The fully connected layer helps to predict the start and end positions of the answer based on the learned representations from the LSTM layer.

In [ ]:
class LSTMModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, num_layers, dropout):
        super(LSTMModel, self).__init__()
        self.lstm = nn.LSTM(input_dim, hidden_dim, num_layers, dropout=dropout, batch_first=True)
        self.fc = nn.Linear(hidden_dim, 2)

    def forward(self, x):
        out, _ = self.lstm(x)
        logits = self.fc(out)

        return logits

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

### <font color='blue'> Padding labels </font>

This function is designed to pad a list of labels to a specified maximum length. This function is used in in this task because the length of the labels may vary across different documents. This method is used to convert training labels to padded_train_labels.

In [ ]:
def pad_labels(labels, max_length):
    padded_labels = []
    for label in labels:
        if len(label) < max_length:
            pad_length = max_length - len(label)
            padded_label = np.pad(label, (0, pad_length), constant_values=0)
            padded_labels.append(padded_label)
        else:
            padded_labels.append(label[:max_length])
    return padded_labels

In [ ]:
max_train_length = max(len(label) for label in train_labels)
padded_train_labels = pad_labels(train_labels, max_train_length)

In [ ]:
padded_train_labels = torch.tensor(padded_train_labels).to(device)

<ipython-input-98-e37d34318493>:1: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ../torch/csrc/utils/tensor_new.cpp:245.)
  padded_train_labels = torch.tensor(padded_train_labels).to(device)


## <font color='green'> Training </font>
In the training part we train our data on the LSTM/CRF model then evaluate it on the test data.

### <font color='blue'> Training with training data </font>

In this part we only use training vectors and without using any batches or Dataloaders. We train the model on the data, two times.

In [ ]:
input_dim = 19151
hidden_dim = 128
num_layers = 4
dropout = 0.2
num_labels = 2

model = LSTMCRFModel(input_dim, hidden_dim, num_labels)
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 2

for epoch in range(num_epochs):
    documents = torch.tensor(training_vectors, dtype=torch.float32).to(device)
    padded_train_labels = torch.tensor(padded_train_labels).to(device)
    optimizer.zero_grad()
    mask = documents.gt(0).float()
    lengths = mask.sum(dim=1).int()
    outputs = model(documents, padded_train_labels, mask=mask)
    loss = -outputs
    loss.backward()
    optimizer.step()
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {loss.item()}")

### <font color='blue'> Training with training and validation data </font>

Unlike the last part, in this section we use validation and training data in the process. We also use Dataloaders and batches so our model be more accurate and more efficient. Also in this part padding labels is used so we can compare the predicted output with the acutal output.

In [ ]:
input_dim = 19151
hidden_dim = 128
num_layers = 4
num_labels = 2
dropout = 0.2

model = LSTMCRFModel(input_dim, hidden_dim, num_labels)
criterion = model.crf.loss
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
num_epochs = 10

total_training_loss = 0
avg_training_loss = 0

num_epochs = 1
for epoch in range(num_epochs):
    model.train()
    training_loss = 0.0
    for batch in train_loader:
        documents, labels = batch
        documents_length = len(documents)
        documents = torch.tensor(documents, dtype=torch.float32).to(device)
        max_length = max(len(label) for label in labels)
        padded_labels = pad_labels(labels, max_length)
        padded_labels = torch.tensor(padded_labels).to(device)
        optimizer.zero_grad()
        mask = documents.gt(0).float()
        lengths = mask.sum(dim=1).int()
        outputs = model(documents, padded_labels, mask=mask)
        loss = -outputs
        optimizer.zero_grad()

        ## for LSTMModel:
        #  documents, start_indices, end_indices = batch
        #  documents = torch.tensor(documents).to(device)
        #  start_indices = torch.tensor(start_indices).to(device)
        #  end_indices = torch.tensor(end_indices).to(device)
        #  documents = documents.to(torch.float32)
        #  start_indices = start_indices.to(torch.float32)
        #  end_indices = end_indices.to(torch.float32)
        #  logits = model(documents)
        #  start_logits = logits[:, 0]
        #  end_logits = logits[:, 1]
        #  start_loss = criterion(start_logits, start_indices)
        #  end_loss = criterion(end_logits, end_indices)
        #  loss = start_loss + end_loss
        #  total_training_loss += loss

        loss.backward()
        optimizer.step()
        training_loss += loss.item() * documents_length

    average_loss = training_loss / len(train_dataset)
    print(f'Epoch {epoch + 1}/{num_epochs}, Training Loss: {average_loss:.4f}')


    model.eval()
    with torch.no_grad():
        total_loss = 0.0
        for batch in valid_loader:

            documents, labels = batch
            documents_length = len(documents)
            documents = torch.tensor(documents, dtype=torch.float32).to(device)
            max_length = max(len(label) for label in labels)
            padded_labels = pad_labels(labels, max_length)
            padded_labels = torch.tensor(padded_labels).to(device)
            mask = documents.gt(0).float()
            lengths = mask.sum(dim=1).int()
            outputs = model(documents, padded_labels, mask=mask)
            loss = -outputs

            ## for LSTMModel:
            #  documents, start_indices, end_indices = batch
            #  documents = torch.tensor(documents).to(device)
            #  start_indices = torch.tensor(start_indices).to(device)
            #  end_indices = torch.tensor(end_indices).to(device)
            #  documents = documents.to(torch.float32)
            #  start_indices = start_indices.to(torch.float32)
            #  end_indices = end_indices.to(torch.float32)
            #  logits = model(documents)
            #  start_logits = logits[:, 0]
            #  end_logits = logits[:, 1]
            #  start_loss = criterion(start_logits, start_indices)
            #  end_loss = criterion(end_logits, end_indices)
            #  loss = start_loss + end_loss
            #  total_training_loss += loss

            total_loss += loss.item() * documents_length

        average_loss = total_loss / len(valid_dataset)
        print(f'Epoch {epoch + 1}/{num_epochs}, Validation Loss: {average_loss:.4f}')

### <font color='blue'> Evaluation </font>

FInally we can evaluate our model on the test data and use different metrics to asses the efficieny and accuracy of that.

In [ ]:
model.eval()
accuracy = 0.0
total = 0.0
test_loss = 0
with torch.no_grad():
    for data in test_loader:
        documents, labels = batch
        documents_length = len(documents)
        documents = torch.tensor(documents, dtype=torch.float32).to(device)
        max_length = max(len(label) for label in labels)
        padded_labels = pad_labels(labels, max_length)
        padded_labels = torch.tensor(padded_labels).to(device)
        mask = documents.gt(0).float()
        lengths = mask.sum(dim=1).int()
        outputs = model(documents, padded_labels, mask=mask)
        loss = -outputs


      # # for LSTMModel:
      #  documents, start_indices, end_indices = batch
      #  documents = torch.tensor(documents).to(device)
      #  start_indices = torch.tensor(start_indices).to(device)
      #  end_indices = torch.tensor(end_indices).to(device)
      #  documents = documents.to(torch.float32)
      #  start_indices = start_indices.to(torch.float32)
      #  end_indices = end_indices.to(torch.float32)
      #  logits = model(documents)
      #  start_logits = logits[:, 0]
      #  end_logits = logits[:, 1]
      #  start_loss = criterion(start_logits, start_indices)
      #  end_loss = criterion(end_logits, end_indices)
      #  loss = start_loss + end_loss
      #  total_training_loss += loss

        total_loss += loss.item() * documents_length
#         _, predicted = torch.max(outputs.data, 1)
#         accuracy += (predicted == labels).sum().item()
#         total += start_indices.size(0)

test_loss /= len(test_loader.dataset)
print(f"Test Loss: {test_loss:.4f}")
accuracy = (100 * accuracy / total)
print(f"accuracy: {accuracy:.2f}")

# <font color='red'> Transformer </font>
----

Available in another notebook. (NLP_HW4_token_clf_transformer)